# 연습문제


In [1]:
import sys
assert sys.version_info >= (3, 7)
from packaging import version
import sklearn

# 1. 투표 기반 분류기


문제: MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눕니다(예를 들면 훈련에 50,000개 샘플, 검증에 10,000개 샘플, 테스트에 10,000개 샘플).

In [2]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True,
                                as_frame=False, parser='auto')

MNIST 데이터셋은 앞서 로드했습니다. 이 데이터셋은 이미 훈련 세트(처음 60,000개의 샘플)와 테스트 세트(마지막 10,000개의 샘플)로 분할되어 있으며, 훈련 세트는 이미 뒤섞여 있습니다. 따라서 처음 50,000개의 샘플을 새 훈련 세트에, 다음 10,000개의 샘플을 검증 세트에, 마지막 10,000개의 샘플을 테스트 세트에 사용하기만 하면 됩니다:

In [3]:
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [4]:
# 데이터 스케일링
# SVM, MLP는 입력데이터의 스케일에 민감함

X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

문제: 그런 다음 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기 같은 여러 종류의 분류기를 훈련시킵니다.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier

In [6]:
# 랜덤포레스트 분류기
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rnd_clf.fit(X_train, y_train)

# 엑스트라 트리
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
ext_clf.fit(X_train, y_train)

# SVM 분류기
linear_svc = LinearSVC(max_iter=1000, random_state=42)
#LinearSVC를 사용하면 확률 메서드가 없어서 오류, 그냥 SVC는 데이터 양이 많아서 오래걸림
svm_clf = CalibratedClassifierCV(linear_svc, cv=3)
svm_clf.fit(X_train, y_train)

# MLP 분류기
mlp_clf = MLPClassifier(random_state=42)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

문제: 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요.

In [7]:
# 개별 분류기 검증 테스트셋 정확도

print("개별 분류기 검증 테스트셋 정확도")
for clf in (rnd_clf, ext_clf, svm_clf, mlp_clf):
    print(clf.__class__.__name__, "=", clf.score(X_valid, y_valid))

개별 분류기 검증 테스트셋 정확도
RandomForestClassifier = 0.9734
ExtraTreesClassifier = 0.9743
CalibratedClassifierCV = 0.9215
MLPClassifier = 0.9776


In [8]:
from sklearn.ensemble import VotingClassifier

# 직접 투표 방법
voting_clf_hard = VotingClassifier(
    estimators=[('rf', rnd_clf), ('ext', ext_clf), ('svm', svm_clf), ('mlp', mlp_clf)],
    voting='hard')
voting_clf_hard.fit(X_train, y_train)

print("직접 투표 방법 검증 테스트셋 정확도")
voting_clf_hard.score(X_valid, y_valid)

직접 투표 방법 검증 테스트셋 정확도


0.9752

In [9]:
# 간접 투표 방법
voting_clf_soft = VotingClassifier(
    estimators=[('rf', rnd_clf), ('ext', ext_clf), ('svm', svm_clf), ('mlp', mlp_clf)],
    voting='soft')
voting_clf_soft.fit(X_train, y_train)

print("간접 투표 방법 검증 테스트셋 정확도")
voting_clf_soft.score(X_valid, y_valid)

간접 투표 방법 검증 테스트셋 정확도


0.9783

문제: 앙상블을 얻고 나면 테스트 세트로 확인해보세요. 개개의 분류기와 비교해서 성능이 얼마나 향상되나요?

In [10]:
# 테스트셋 정확도
print("개별 분류기 정확도")
for clf in (rnd_clf, ext_clf, svm_clf, mlp_clf):
    print(clf.__class__.__name__, "=", clf.score(X_test, y_test))

print("직접 투표 방법 정확도")
print(voting_clf_hard.score(X_test, y_test))

print("간접 투표 방법 정확도")
print(voting_clf_soft.score(X_test, y_test))

개별 분류기 정확도
RandomForestClassifier = 0.968
ExtraTreesClassifier = 0.9703
CalibratedClassifierCV = 0.9168
MLPClassifier = 0.9784
직접 투표 방법 정확도
0.9706
간접 투표 방법 정확도
0.9768


# 2. 스태킹 앙상블

문제: 이전 연습문제의 각 분류기를 실행해서 검증 세트에서 예측을 만들고 그 결과로 새로운 훈련 세트를 만들어보세요. 각 훈련 샘플은 하나의 이미지에 대한 전체 분류기의 예측을 담은 벡터이고 타깃은 이미지의 클래스입니다. 새로운 훈련 세트에 분류기 하나를 훈련시켜보세요.

In [11]:
import numpy as np

clf = [rnd_clf, ext_clf, svm_clf, mlp_clf]

X_valid_pred = np.empty((len(X_valid), len(clf)), dtype=np.float32) # (검증 세트 샘플 수 X 모델 개수) 크기의 빈 배열
X_test_pred = np.empty((len(X_test), len(clf)), dtype=np.float32)

for index, clf in enumerate(clf):
    # 각 모델이 예측한 결과(클래스)를 저장
    X_valid_pred[:, index] = clf.predict(X_valid)
    X_test_pred[:, index] = clf.predict(X_test)

# 랜덤포레스트를 활용해 새로운 훈련세트 + 훈련
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_valid_pred, y_valid)

# 성능 확인
print("새로운 훈련 세트 분류 성능")
rnd_forest_blender.score(X_test_pred, y_test)


새로운 훈련 세트 분류 성능


0.9739

문제: 이제 StackingClassifier를 사용하여 다시 시도해 보세요. 성능이 더 좋아졌나요? 그렇다면 그 이유는 무엇인가요?

StackingClassifier는 K-겹 교차 검증을 사용하기 때문에 별도의 검증 세트가 필요하지 않으므로 훈련 세트와 검증 세트를 더 큰 훈련 세트로 합쳐보겠습니다:

In [12]:
X_train_full, y_train_full = X_mnist[:60_000], y_mnist[:60_000]

#데이터 스케일링
X_train_full = X_train_full / 255.0

In [14]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[('rf', rnd_clf), ('ext', ext_clf), ('svm', svm_clf), ('mlp', mlp_clf)],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5, # 교차 검증 폴드 수
    n_jobs=-1
)
stacking_clf.fit(X_train_full, y_train_full)

# 성능 확인
print("스태킹 앙상블 분류 성능")
stacking_clf.score(X_test, y_test)

/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


스태킹 앙상블 분류 성능


0.9814